Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
Определить параметры которые влияют на "точность" определения вектора t
Использовать решение на базе нейронных сетей. 
Любые идеи. 

Есть видео полета дрона, анализируем с помощью детектора sift.
Выводим траекторию на трехмерный график.
Результаты сохраняем в таблице csv


In [11]:

# загрузка необходимых библиотек и двух изображений
import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter_matches_distance(matches, dist_threshold):
    """
    Фильтрация совпавших особенностей из двух изображений по расстоянию между лучшими совпадениями

    Аргументы:
    match -- список совпавших особенностей из двух изображений
    dist_threshold -- максимальное допустимое относительное расстояние между лучшими совпадениями, (0.0, 1.0)

    Возвращает:
    filtered_match -- список хороших совпадений, удовлетворяющих порогу расстояния
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_match.append(m)

    return filtered_match


In [12]:
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    """
    Сопоставление особенностей из двух изображений

    Аргументы:
    des1 -- список дескрипторов ключевых точек на первом изображении
    des2 -- список дескрипторов ключевых точек на втором изображении
    matching -- (str) может быть 'BF' для Brute Force или 'FLANN'
    detector -- (str) может быть 'sift' или 'orb'. По умолчанию 'sift'
    sort -- (bool) сортировать ли совпадения по расстоянию. По умолчанию True
    k -- (int) количество соседей для сопоставления каждой особенности

    Возвращает:
    matches -- список совпадений из двух изображений. Каждое match[i] содержит k или менее совпадений для 
               того же дескриптора запроса

    """
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

Загружаем видео и считываем кадры с изменяемым интервалом

In [13]:
video_path = 'drone.mp4'  # имя файла (и путь, если необходимо)
myvid = cv2.VideoCapture(video_path) # загрузка в cv2
# Проверка успешного открытия видео
if not myvid.isOpened():
    print("Видео файл не найден")
else:
    print("Видео открыто") 

# матрица камеры K
K = np.array([[3000,   0.    , 960],
              [  0.    , 3000 , 540 ],
              [  0.    ,   0.    ,   1.]], dtype=np.float32)

sift = cv2.SIFT_create()
poses = [] # список для поз камеры
# Список для позиций камеры
positions = [np.array([0, 0, 0])]

# Чтение первого кадра видео
ret, frame = myvid.read()
img1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
i = 0
interval = 20 # интервал между рассматриваемыми кадрами, например, interval = 5 для каждого 5-го кадра
while True: # i < 1000:
    i += 1
    ret, frame = myvid.read() # чтение следующего кадра          
    if ret and i % interval == 0: # каждый 10-й кадр
        print(f'Текущий кадр: {i}')
        img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        try:
            keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
            keypoints2, descriptors2 = sift.detectAndCompute(img2, None)      
            matches = match_features(descriptors1, descriptors2, matching='BF', detector='sift', sort=True)
            matches = filter_matches_distance(matches, 0.7)
            # matches = matcher.match(np.array(descriptors1, dtype=np.uint8), np.array(descriptors2, dtype=np.uint8))
            pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
       
            F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.LMEDS, prob=0.999, threshold=1.0) # LMEDS    
            # F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0) #RANSAC    
            a, R, t, b = cv2.recoverPose(F, pts1, pts2, K, mask)
            # print(R)
            poses.append((R, t))

            # Обновить позицию камеры
            current_position = positions[-1]
            new_position = current_position + np.dot(R, t).T[0]
            positions.append(new_position)
            # Сохранение текущего кадра как предыдущего
            img1 = img2 
        except:
            pass
    elif ret and i % interval != 0:
        pass
    else:
        print('Видео - всё')
        myvid.release()          # освобождение видеоканала после завершения просмотра
        # cv2.destroyAllWindows()  # закрытие всех окон
        break                    # выход из цикла


Видео открыто
Текущий кадр: 5
Текущий кадр: 10
Текущий кадр: 15
Текущий кадр: 20
Текущий кадр: 25
Текущий кадр: 30
Текущий кадр: 35
Текущий кадр: 40
Текущий кадр: 45
Текущий кадр: 50
Текущий кадр: 55
Текущий кадр: 60
Текущий кадр: 65
Текущий кадр: 70
Текущий кадр: 75
Текущий кадр: 80
Текущий кадр: 85
Текущий кадр: 90
Текущий кадр: 95
Текущий кадр: 100
Текущий кадр: 105
Текущий кадр: 110
Текущий кадр: 115
Текущий кадр: 120
Текущий кадр: 125
Текущий кадр: 130
Текущий кадр: 135
Текущий кадр: 140
Текущий кадр: 145
Текущий кадр: 150
Текущий кадр: 155
Текущий кадр: 160
Текущий кадр: 165
Текущий кадр: 170
Текущий кадр: 175
Текущий кадр: 180
Текущий кадр: 185
Текущий кадр: 190
Текущий кадр: 195
Текущий кадр: 200
Текущий кадр: 205
Текущий кадр: 210
Текущий кадр: 215
Текущий кадр: 220
Текущий кадр: 225
Текущий кадр: 230
Текущий кадр: 235
Текущий кадр: 240
Текущий кадр: 245
Текущий кадр: 250
Текущий кадр: 255
Текущий кадр: 260
Текущий кадр: 265
Текущий кадр: 270
Текущий кадр: 275
Текущий кадр: 28

Визуализируем результат


In [14]:
print(poses)

[(array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 5.55111512e-17],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), array([[-0.57735027],
       [ 0.57735027],
       [-0.57735027]])), (array([[ 9.99999919e-01,  4.00353278e-04,  2.87767758e-05],
       [-4.00359094e-04,  9.99999899e-01,  2.02386323e-04],
       [-2.86957469e-05, -2.02397828e-04,  9.99999979e-01]]), array([[-0.91789831],
       [-0.24661532],
       [ 0.31087549]])), (array([[ 1.00000000e+00,  1.44193082e-05,  1.81151760e-05],
       [-1.44193153e-05,  1.00000000e+00,  3.91494893e-07],
       [-1.81151703e-05, -3.91756102e-07,  1.00000000e+00]]), array([[-0.1336461 ],
       [-0.03380801],
       [-0.99045229]])), (array([[ 9.99996359e-01,  1.35177279e-04, -2.69517628e-03],
       [-1.35218019e-04,  9.99999991e-01, -1.49336570e-05],
       [ 2.69517424e-03,  1.52980390e-05,  9.99996368e-01]]), array([[ 0.98424543],
       [-0.09674965],
       [ 0.14798794]])), 

In [15]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory),np.array(pose_cam)

In [16]:
trajectory, pose_cam = create_trajectory(poses)

In [17]:
import plotly.graph_objects as go
import numpy as np
# Создание фигуры
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=trajectory[1:, 0],
    y=trajectory[1:, 1],
    z=trajectory[1:, 2],
    mode='lines+markers',
    marker=dict(size=3, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam,trajectory)):
    # Направление камеры (ось Z камеры)
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    # Добавляем линию, представляющую направление камеры
    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene = dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()

Мы использовали детектор SIFT для анализа видео и расчета 3D-траектории движения дрона и ориентации его камеры. Для нахождения эссенциальной матрицы применялись методы cv2.LMEDS и cv2.RANSAC. Точность расчета зависит от интервала между обрабатываемыми кадрами и метода расчета эссенциальной матрицы. Например, при анализе каждого пятого кадра траектория выглядела достаточно плавной.

---